In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
from joblib import dump


In [4]:
flight_df = pd.read_csv('flight.csv', header=0, encoding='cp949')
weather_df = pd.read_csv('weather.csv', header=0, encoding='cp949')

print('항공 데이터 컬럼: ', flight_df.columns)
print(flight_df.head())

print('날씨 데이터 컬럼: ', weather_df.columns)
print(weather_df.head())


항공 데이터 컬럼:  Index(['flight_info_id', 'departure_date', 'airline', 'flight_code',
       'destination', 'departure_time_plan', 'departure_time_expexted',
       'departure_time_real', 'division', 'flight_status', 'cause'],
      dtype='object')
   flight_info_id  departure_date airline flight_code destination  \
0               1        20070101  중국남방항공      CZ3086   HAK(하이커우)   
1               2        20070101    대한항공      KE1452     GMP(김포)   
2               3        20070101  중국남방항공      CZ3088     SYX(산야)   
3               4        20070101    대한항공       KE787   FUK(후쿠오카)   
4               5        20070101    대한항공       KE621    MNL(마닐라)   

  departure_time_plan departure_time_expexted departure_time_real division  \
0               06:30                   06:49               06:34       여객   
1               06:30                   06:30               06:50      NaN   
2               07:00                   07:23               07:10       여객   
3               08:00        

지점 point
일시 weather_date
기온 temperature
10분평균풍속(KT) wind_speed_10m_avg_kt (o) -> WS10 * 
10분평균풍향(deg) wind_dir_10m_avg_deg (o) -> WD10
10분평균MOR(m) mor_10m_avg_m (x)
10분평균RVR(m) rvr_10m_avg_m (x)
누적강수량(mm) cumulative_precipitation_mm (x)

In [8]:
wdf = weather_df.copy()

wdf['mor_10m_avg_km'] = wdf['mor_10m_avg_m'] / 1000
wdf['weather_date'] = pd.to_datetime(wdf['weather_date'])

wdf.drop(columns=['weather_id', 'point', 'wind_dir_10m_avg_deg', 'rvr_10m_avg_m', 'mor_10m_avg_m'],
         inplace=True)

wdf.describe()

,weather_date,temperature,wind_speed_10m_avg_kt,cumulative_precipitation_mm,mor_10m_avg_km
count,8801013,8.800550e+06,8.800920e+06,8.775057e+06,8.801012e+06
mean,2015-05-17 05:19:54.027431936,1.222269e+01,7.324073e+00,1.537805e+00,4.099915e+00
min,2007-01-01 00:01:00,-9.999000e+02,-9.999000e+02,-9.999000e+02,-9.999000e+00
25%,2011-02-18 15:36:00,3.700000e+00,4.100000e+00,0.000000e+00,3.000000e+00
50%,2015-03-30 06:58:00,1.340000e+01,6.600000e+00,0.000000e+00,3.000000e+00
75%,2019-06-05 19:48:00,2.160000e+01,9.700000e+00,0.000000e+00,3.000000e+00
max,2024-04-02 00:00:00,3.680000e+01,5.120000e+01,1.765000e+02,1.000000e+01
std,NaN,1.881518e+01,8.368988e+00,1.269778e+01,2.678412e+00


['flight_info_id', 'departure_date', 'airline', 'flight_code',
       'destination', 'departure_time_plan', 'departure_time_expexted',
       'departure_time_real', 'division', 'flight_status', 'cause']
   flight_info_id  departure_date airline flight_code destination  \
0               1        20070101  중국남방항공      CZ3086   HAK(하이커우)   
1               2        20070101    대한항공      KE1452     GMP(김포)   
2               3        20070101  중국남방항공      CZ3088     SYX(산야)   
3               4        20070101    대한항공       KE787   FUK(후쿠오카)   
4               5        20070101    대한항공       KE621    MNL(마닐라)   

  departure_time_plan departure_time_expexted departure_time_real division  \
0               06:30                   06:49               06:34       여객   
1               06:30                   06:30               06:50      NaN   
2               07:00                   07:23               07:10       여객   
3               08:00                   08:14               08:10       여객   
4               08:00                   08:20               08:16       여객   

  flight_status cause  
0            출발   NaN  
1            출발   NaN  
2            출발   NaN  
3            출발   NaN  
4            출발   NaN 

In [24]:
fdf = flight_df.copy()
fdf = fdf[fdf['flight_status'] == '지연']
fdf = fdf[fdf['division'] == '여객']

fdf['departure_date'] = pd.to_datetime(fdf['departure_date'], format='%Y%m%d')
fdf['departure_time_plan'] = pd.to_datetime(fdf['departure_time_plan'], format='%H:%M',
                                            errors='coerce')
fdf['departure_time_real'] = pd.to_datetime(fdf['departure_time_real'], format='%H:%M',
                                            errors='coerce')

fdf['airline'] = fdf['flight_code'].str[:2]

fdf.dropna(subset=['departure_time_plan'], inplace=True)
fdf.dropna(subset=['departure_time_real'], inplace=True)
fdf.dropna(subset=['departure_date'], inplace=True)

fdf['cause'] = fdf['cause'].fillna('기타')
fdf['destination'] = fdf['destination'].str[:3]

fdf['departure_time'] = pd.to_datetime(
        fdf['departure_date'].dt.strftime('%Y-%m-%d') + ' ' + fdf[
            'departure_time_plan'].dt.strftime('%H:%M'))

departure_datetime = pd.to_datetime(
        fdf['departure_date'].dt.strftime('%Y-%m-%d') + ' ' + fdf[
            'departure_time_real'].dt.strftime('%H:%M'))
delay_seconds = (departure_datetime - fdf['departure_time']).dt.total_seconds()
fdf['delay_minute'] = delay_seconds.apply(lambda x: x if x > 0 else 86400 + x).astype(int) / 60

# fdf['delay_minute'] = (pd.to_datetime(
#         fdf['departure_date'].dt.strftime('%Y-%m-%d') + ' ' + fdf[
#             'departure_time_real'].dt.strftime(
#                 '%H:%M')) - fdf['departure_time']).dt.total_seconds().astype(int) / 60
fdf.drop(columns=['flight_info_id', 'division', 'flight_status', 'departure_date',
                  'departure_time_plan', 'departure_time_real', 'departure_time_expexted'],
         inplace=True)

fdf.head()

,airline,flight_code,destination,cause,departure_time,delay_minute
76,KE,KE831,SHE,기상-시정,2007-01-01 11:15:00,85.0
91,CA,CA172,TSN,연결-항공기,2007-01-01 12:20:00,126.0
99,MU,MU5022,XIY,연결-항공기,2007-01-01 12:45:00,77.0
143,MU,MU2034,TAO,기타,2007-01-01 15:10:00,70.0
185,CA,CA126,PEK,연결-항공기,2007-01-01 18:20:00,83.0


In [28]:
def normalize_date(row):
    year = row['departure_time'].year
    month = row['departure_time'].month
    day = row['departure_time'].day
    is_leap_year = row['departure_time'].is_leap_year

    month_days = [0, 31, 29 if is_leap_year else 28, 31, 30, 31, 30, 31, 31,
                  30,
                  31, 30, 31]

    cumulative_days = sum(month_days[:month]) + day

    max_days = 366 if is_leap_year else 365

    normalized = (cumulative_days - 1) / (
            max_days - 1)  # 1월 1일은 0으로, 12월 31일은 1로 정규화
    return normalized * 10

In [46]:
merged_df = pd.merge(fdf, wdf, left_on='departure_time', right_on='weather_date', how='inner')

merged_df['term'] = merged_df.apply(normalize_date, axis=1)

merged_df.drop(columns=['departure_time', 'weather_date'], inplace=True)
merged_df.head()

,airline,flight_code,destination,cause,delay_minute,temperature,wind_speed_10m_avg_kt,cumulative_precipitation_mm,mor_10m_avg_km,term
0,KE,KE831,SHE,기상-시정,85.0,3.5,5.2,0.0,3.0,0.0
1,CA,CA172,TSN,연결-항공기,126.0,4.2,4.9,0.0,3.0,0.0
2,MU,MU5022,XIY,연결-항공기,77.0,4.4,3.7,0.0,3.0,0.0
3,MU,MU2034,TAO,기타,70.0,5.1,3.1,0.0,3.0,0.0
4,CA,CA126,PEK,연결-항공기,83.0,5.6,4.3,0.0,3.0,0.0


In [39]:
categorical_features = ['airline', 'flight_code', 'destination', 'cause']
numeric_features = [col for col in merged_df.columns if col not in categorical_features]

encoder = OneHotEncoder(handle_unknown='ignore')
column_transformer = ColumnTransformer([('encoder', encoder, categorical_features)],
                                       remainder='passthrough')
encoded_data = column_transformer.fit_transform(merged_df)

dump(column_transformer, "one_hot_encoder.pkl")

encoded_df = pd.DataFrame(encoded_data.toarray())
new_column_names = column_transformer.named_transformers_['encoder'].get_feature_names_out(
        input_features=categorical_features)
all_column_names = list(new_column_names) + list(numeric_features)
encoded_df.columns = all_column_names

encoded_df.head()

,airline_2P,airline_3P,airline_3U,airline_5J,airline_5M,airline_5W,airline_5X,airline_5Y,airline_6K,airline_7C,...,"cause_환승 수속 오류(승객, 위탁 수화물)",cause_환자,cause_활주로변경,cause_활주로폐쇄,delay_minute,temperature,wind_speed_10m_avg_kt,cumulative_precipitation_mm,mor_10m_avg_km,term
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,85.0,3.5,5.2,0.0,3.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,126.0,4.2,4.9,0.0,3.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,77.0,4.4,3.7,0.0,3.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,70.0,5.1,3.1,0.0,3.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,83.0,5.6,4.3,0.0,3.0,0.0


In [40]:
scaled_df = encoded_df.copy()
scaled_df.dropna(inplace=True)
scaled_df.to_csv('scaled.csv', index=False)

In [42]:
Q1 = scaled_df['delay_minute'].quantile(0.25)
Q3 = scaled_df['delay_minute'].quantile(0.75)
IQR = Q3 - Q1
not_outlier_condition = (scaled_df['delay_minute'] >= (Q1 - 1.5 * IQR)) & (
        scaled_df['delay_minute'] <= (Q3 + 1.5 * IQR))

scaled_df = scaled_df[not_outlier_condition]

In [43]:
X = scaled_df.drop('delay_minute', axis=1)
y = scaled_df['delay_minute']

print(f"Delayed avg(minute): {scaled_df["delay_minute"].sum() / len(scaled_df["delay_minute"])}")

Delayed avg(minute): 73.78613221802868


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lasso_cv = make_pipeline(StandardScaler(), LassoCV(cv=5))
lasso_cv.fit(X_train, y_train)

y_pred = lasso_cv.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 16.47059218005207


In [45]:
dump(lasso_cv, 'lasso_cv_model.joblib')

['lasso_cv_model.joblib']